In [22]:
%matplotlib inline

import os
import re
import csv
import math
import gzip
import glob
import pickle
import datetime
import warnings
import numpy as np
import netCDF4
import matplotlib.pyplot as pl
from dateutil import parser
from datetime import datetime, timedelta
from matplotlib.dates import MonthLocator, DateFormatter, WeekdayLocator

from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go

warnings.filterwarnings('ignore')

In [23]:
def f_nbin(t, step=1):
    return int(np.round((np.nanmax(t)-np.nanmin(t))/step))

def get_datetime_from_filename(fname):
    dtstr = re.findall('[0-9]{8}', fname)[0]
    the_date = datetime.strptime(dtstr, '%Y%m%d')
    
    return the_date

In [29]:
def msgr_read(flist, max_value):
    
    # ref1 PR reflectivity
    # ref2 GR reflectivity
    # ref3 PR reflec S-band, snow
    # ref4 PR reflec S-band, hail
    # ref5 GR reflectivity Ku-band
    
    refl_offset    = []
    refl_diff_list = []
    refl_std       = []
    dtime          = []
    sample_size    = []
    
    for fd in flist:
        
        try:
            #use only first pass. Using second pass becomes more difficult when collating refl diff samples
            pass_no = fd[-4]
            if pass_no == '2':
                continue

            with netCDF4.Dataset(fd, "r") as ncid:

                #read data from file
                date     = parser.parse(re.findall("[0-9]{8}", fd)[0])
                z        = ncid['z'][:]
                zbb      = ncid['zbb'][:]
                zbbw     = ncid['bbwidth'][:]
                ref_GR   = ncid['ref2'][:]  #GR in dB
                ref_SR_S = ncid['ref3'][:]  #SR (Sband/Snow) in dB
                nrej_GR  = ncid['nrej1'][:]
                nrej_SR  = ncid['nrej2'][:]
                ntot_GR  = ncid['ntot1'][:]
                ntot_SR  = ncid['ntot2'][:]

            #apply same masking as matchvol to recalculate means as required
            pos         = np.logical_and(z > (zbb - zbbw/2), z < (zbb + zbbw/2)) | (np.isnan(ref_GR)) | (np.isnan(ref_SR_S)) | (ref_GR < 21) | (ref_SR_S < 21) | (nrej_GR/ntot_GR > 0.3) | (nrej_SR/ntot_SR > 0.3)
            if not np.any(~pos):
                continue
            ref_GR[pos]   = np.nan
            ref_SR_S[pos] = np.nan

            ref_diff = ref_GR - ref_SR_S
            ref_diff = ref_diff[(ref_diff <= np.nanpercentile(ref_diff,95)) & (ref_diff >= np.nanpercentile(ref_diff,5))]

            #gerneate and append stats
            offset_val = np.nanmean(ref_diff)

            #check for limit on reflectivity offet, skip if it exceeds this limit
            if abs(offset_val) > max_value:
                continue

            refl_offset.append(offset_val)
            refl_diff_list.append(ref_diff)
            refl_std.append(np.nanstd(ref_diff))        
            sample_size.append(len(ref_diff))
            dtime.append(date)
        except Exception as e:
            print('failed on', fd)
            print(e)
            continue

    #convert to array
    refl_offset = np.array(refl_offset)
    refl_std    = np.array(refl_std)
    sample_size = np.array(sample_size)
    dtime_ku    = np.array(dtime)

    return refl_offset, refl_diff_list, dtime_ku, refl_std, sample_size

In [32]:
msgr_root_path = '/g/data/kl02/jss548/bonn-radar-cal/msgr'
out_root_path  = '/g/data/kl02/jss548/bonn-radar-cal/plotly'
start_yr       = 2014
end_yr         = 2019
max_msgr_value = 10 #used to filter msgr
rid            = 'BOXPOL'

###########################################################################
# MSGR DATA LOAD
###########################################################################
print('loading msgr data for', rid)
#read msgr_data
msgr_path            = '/'.join([msgr_root_path])
msgr_file_list       = sorted(glob.glob(msgr_path + '/*.nc'))
if len(msgr_file_list) > 0:
    #process data
    msgr_refl_offset, msgr_diff_list, msgr_dtime, msgr_refl_std, msgr_sample_size = msgr_read(msgr_file_list, max_msgr_value)
    #plot flag
    msgr_plot = True
else:
    msgr_plot = False
    print('no msgr files found')

#calc total mean
msgr_total_mean = np.round(np.nanmean(msgr_refl_offset), decimals = 2)

###########################################################################
# PLOTLY
###########################################################################

#update user
display('Plotting for ' + rid)

#init plotly
fig      = tools.make_subplots(rows=1, cols=1, shared_xaxes=True)
datemin  = np.datetime64(str(start_yr), 'Y')
datemax  = np.datetime64(str(end_yr), 'Y') + np.timedelta64(1, 'Y')

###########################################################################
# MSGR PLOTLY
###########################################################################        

if msgr_plot:
    trace_msgr = go.Scatter(
                            name='MSGR plot',
                            x=msgr_dtime,
                            y=np.round(msgr_refl_offset, decimals = 2),
                            mode='markers',
                            marker=dict(
                                color = msgr_sample_size,
                                colorscale='YlOrRd',
                                cmax=1000,
                                cauto=False,
                                colorbar=dict(len=0.2, y=0.95, titleside='right',title='No. Samples'),
                                reversescale=True,
                                showscale=True,
                                size=8
                                ),
                            error_y=dict(
                                type='data',
                                symmetric=True,
                                array=np.round(msgr_refl_std, decimals = 2),
                                color='#000000',
                                thickness=1
                                )
                            )


    trace_msgr_mean = go.Scatter(
        name='MSGR Mean',
        x=[datemin, datemax],
        y=[msgr_total_mean, msgr_total_mean],
        line = dict(
            color = ('rgb(22, 96, 167)'),
            width = 2,
            dash = 'dash')
            )
    trace_msgr_breaks = go.Scatter(
        name='MSGR Mean',
        x=[datemin, datemin, np.nan, datemax, datemax, np.nan],
        y=[-100, 100, np.nan, -100, 100, np.nan],
        line = dict(
            color = ('rgb(22, 96, 167)'),
            width = 1,
            dash = 'solid')
            )

    fig.append_trace(trace_msgr, 1, 1)
    fig.append_trace(trace_msgr_mean, 1, 1)
    fig.append_trace(trace_msgr_breaks, 1, 1)

    y_lim       = 5
    if msgr_plot:
        msgr_absmax = math.ceil(np.nanmax(np.abs(msgr_refl_offset)))
        if msgr_absmax >= y_lim:
            y_lim = msgr_absmax + 1



    fig['layout']['xaxis1'].update(title='',
                            range=[datemin, datemax],
                            )
    fig['layout']['yaxis1'].update(title='MSGR Difference (dB)',
                            range=[-y_lim, y_lim],
                            )
###############
#legend
fig['layout']['legend'].update(orientation="h")

#output
out_fn   = '_'.join(['cal_rca_msgr', rid, str(start_yr), str(end_yr)]) + '.html'
plotly_ffn = '/'.join([out_root_path, out_fn])
#save
try:
    py.plot(fig, filename = plotly_ffn)
    print('processed: ', out_fn)
except:
    print('FAILED FOR: ', out_fn)


print('done')
        

loading msgr data for BOXPOL


'Plotting for BOXPOL'

This is the format of your plot grid:
[ (1,1) x1,y1 ]

processed:  cal_rca_msgr_BOXPOL_2014_2019.html
done
